# Elementary Hypothesis Testing

Jaynes introduces the concept of binary hypothesis testing with the example of quality assurance in a factory.

Suppose we have 

IMAGE?

In [47]:
import math
import numpy as np
import pymc3 as pm
from enum import Enum

In [8]:
class Hypothesis(Enum):
    A = 0
    B = 1
    
def prob_data(data, hyp):
    """
    The probability of picking a sequence of x good widgets inside a box with N total widgets
    """
    N, sample = data
    x = sample[sample == 1].size
    
    if hyp == Hypothesis.A:
        p = 2/3
        return Binomial.pmf(x, N, p)
    elif hyp == Hypothesis.A:
        p = 5/6
        return Binomial.pmf(x, N, p)
        
def evidence(hyp, data):
    not_hyp = [h for h in Hypothesis if h is not hyp]
    prior_ev = 10 * math.log()



In [20]:
hyp_prior = Categorical.dist([1/11, 10/11])
hyp_prior.logp(0).eval()

array(-2.39789527)

In [58]:
def binomial(x, n, p):
    coefficient = math.factorial(n) / (math.factorial(x) * math.factorial(n - x))
    return coefficient * pow(p, x) * (1 - p, n - x)

def ev_a(data):
    N = data.size
    num_bad = data[data == 0].size
    num_good = data[data == 1].size
    p_prior = 1/11
    ev_prior = 10 * math.log(p_prior / (1 - p_prior), 10)
    ev_posterior = 10 * (math.log(pow(2/3, num_good) / pow(5/6, num_good), 10) + math.log(pow(1/3, num_bad) / pow(1/6, num_bad), 10))
    return ev_prior + ev_posterior
    
def ev_b(data):
    N = data.size
    num_bad = data[data == 0].size
    num_good = data[data == 1].size
    p_prior = 10/11
    ev_prior = 10 * math.log(p_prior / (1 - p_prior), 10)
    ev_posterior = 10 * (math.log(pow(2/3, num_good) / pow(5/6, num_good), 10) + math.log(pow(1/3, num_bad) / pow(1/6, num_bad), 10))
    return ev_prior + ev_posterior

In [59]:
ev_b(np.array([]))

9.999999999999998